## Import Libraries

In [1]:
import boto3
import sagemaker
import warnings, requests, zipfile, io
import pandas as pd
import os
warnings.simplefilter('ignore')

D:\Python312\Lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml


## Import the Data

In [2]:
zip_file = "2023-us-flights-delay.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

## Exploratory Data Analysis

### US Flights Dataset

In [3]:
flight_df = pd.read_csv('2023-us-flights-delay/US_flights_2023.csv')

In [4]:
flight_df.shape

(6743404, 24)

In [5]:
flight_df.dtypes

FlightDate            object
Day_Of_Week            int64
Airline               object
Tail_Number           object
Dep_Airport           object
Dep_CityName          object
DepTime_label         object
Dep_Delay              int64
Dep_Delay_Tag          int64
Dep_Delay_Type        object
Arr_Airport           object
Arr_CityName          object
Arr_Delay              int64
Arr_Delay_Type        object
Flight_Duration        int64
Distance_type         object
Delay_Carrier          int64
Delay_Weather          int64
Delay_NAS              int64
Delay_Security         int64
Delay_LastAircraft     int64
Manufacturer          object
Model                 object
Aicraft_age            int64
dtype: object

In [6]:
flight_df.head(5)

,FlightDate,Day_Of_Week,Airline,Tail_Number,Dep_Airport,Dep_CityName,DepTime_label,Dep_Delay,Dep_Delay_Tag,Dep_Delay_Type,...,Flight_Duration,Distance_type,Delay_Carrier,Delay_Weather,Delay_NAS,Delay_Security,Delay_LastAircraft,Manufacturer,Model,Aicraft_age
0,2023-01-02,1,Endeavor Air,N605LR,BDL,"Hartford, CT",Morning,-3,0,Low <5min,...,56,Short Haul >1500Mi,0,0,0,0,0,CANADAIR REGIONAL JET,CRJ,16
1,2023-01-03,2,Endeavor Air,N605LR,BDL,"Hartford, CT",Morning,-5,0,Low <5min,...,62,Short Haul >1500Mi,0,0,0,0,0,CANADAIR REGIONAL JET,CRJ,16
2,2023-01-04,3,Endeavor Air,N331PQ,BDL,"Hartford, CT",Morning,-5,0,Low <5min,...,49,Short Haul >1500Mi,0,0,0,0,0,CANADAIR REGIONAL JET,CRJ,10
3,2023-01-05,4,Endeavor Air,N906XJ,BDL,"Hartford, CT",Morning,-6,0,Low <5min,...,54,Short Haul >1500Mi,0,0,0,0,0,CANADAIR REGIONAL JET,CRJ,17
4,2023-01-06,5,Endeavor Air,N337PQ,BDL,"Hartford, CT",Morning,-1,0,Low <5min,...,50,Short Haul >1500Mi,0,0,0,0,0,CANADAIR REGIONAL JET,CRJ,10


In [7]:
flight_df.describe()

,Day_Of_Week,Dep_Delay,Dep_Delay_Tag,Arr_Delay,Flight_Duration,Delay_Carrier,Delay_Weather,Delay_NAS,Delay_Security,Delay_LastAircraft,Aicraft_age
count,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06,6.743404e+06
mean,3.982793e+00,1.220099e+01,3.790013e-01,6.627152e+00,1.402978e+02,5.169804e+00,7.428505e-01,2.566957e+00,3.064876e-02,5.681108e+00,1.348063e+01
std,2.001762e+00,5.507936e+01,4.851385e-01,5.707892e+01,7.287216e+01,3.645732e+01,1.435393e+01,1.500484e+01,1.628923e+00,3.044647e+01,7.891495e+00
min,1.000000e+00,-9.900000e+01,0.000000e+00,-1.190000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.000000e+00,-5.000000e+00,0.000000e+00,-1.500000e+01,8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00
50%,4.000000e+00,-2.000000e+00,0.000000e+00,-6.000000e+00,1.240000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.200000e+01
75%,6.000000e+00,9.000000e+00,1.000000e+00,9.000000e+00,1.710000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01
max,7.000000e+00,4.413000e+03,1.000000e+00,4.405000e+03,7.950000e+02,3.957000e+03,1.860000e+03,1.708000e+03,1.460000e+03,3.581000e+03,5.700000e+01


In [8]:
flight_df.isnull().sum()

FlightDate            0
Day_Of_Week           0
Airline               0
Tail_Number           0
Dep_Airport           0
Dep_CityName          0
DepTime_label         0
Dep_Delay             0
Dep_Delay_Tag         0
Dep_Delay_Type        0
Arr_Airport           0
Arr_CityName          0
Arr_Delay             0
Arr_Delay_Type        0
Flight_Duration       0
Distance_type         0
Delay_Carrier         0
Delay_Weather         0
Delay_NAS             0
Delay_Security        0
Delay_LastAircraft    0
Manufacturer          0
Model                 0
Aicraft_age           0
dtype: int64

#### Use data from Los Angeles International Airport (LAX) for this project

In [9]:
lax_df = flight_df[flight_df['Dep_Airport'] == 'LAX']
lax_df.shape

(192260, 24)

In [10]:
lax_df.columns = lax_df.columns.str.lower()
print(lax_df)

         flightdate  day_of_week                 airline tail_number  \
16668    2023-01-01            7  American Airlines Inc.      N101NN   
16669    2023-01-02            1  American Airlines Inc.      N116AN   
16670    2023-01-03            2  American Airlines Inc.      N107NN   
16671    2023-01-04            3  American Airlines Inc.      N102NN   
16672    2023-01-05            4  American Airlines Inc.      N110AN   
...             ...          ...                     ...         ...   
6743153  2023-12-31            7         JetBlue Airways      N794JB   
6743262  2023-12-31            7         JetBlue Airways      N991JT   
6743292  2023-12-31            7         JetBlue Airways      N961JT   
6743316  2023-12-31            7         JetBlue Airways      N979JT   
6743329  2023-12-31            7         JetBlue Airways      N988JT   

        dep_airport     dep_cityname deptime_label  dep_delay  dep_delay_tag  \
16668           LAX  Los Angeles, CA       Morning     

### Weather By Airport Dataset

In [11]:
weather_df = pd.read_csv('2023-us-flights-delay/weather_meteo_by_airport.csv')

In [12]:
weather_df.shape

(132860, 10)

In [13]:
weather_df.dtypes

time           object
tavg          float64
tmin          float64
tmax          float64
prcp          float64
snow          float64
wdir          float64
wspd          float64
pres          float64
airport_id     object
dtype: object

In [14]:
weather_df.head(5)

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,pres,airport_id
0,2023-01-01,8.1,2.2,11.7,0.0,0.0,278.0,9.7,1013.8,ABE
1,2023-01-02,5.4,0.0,11.7,0.0,0.0,353.0,3.6,1019.6,ABE
2,2023-01-03,8.4,7.2,9.4,15.2,0.0,50.0,5.0,1013.9,ABE
3,2023-01-04,11.1,6.7,17.2,0.0,0.0,302.0,4.7,1009.8,ABE
4,2023-01-05,12.7,6.7,14.4,7.9,0.0,292.0,7.2,1013.0,ABE


In [15]:
weather_df.describe()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,pres
count,132860.000000,132860.000000,132860.000000,132860.000000,132860.000000,132860.000000,132860.000000,132860.000000
mean,14.042641,8.699833,19.617149,2.520604,6.810131,188.686042,12.424556,1015.650368
std,10.672859,10.680441,11.392223,7.915755,50.935283,114.629099,6.211261,6.944599
min,-39.100000,-76.000000,-35.000000,0.000000,0.000000,0.000000,0.000000,964.300000
25%,6.300000,1.100000,11.700000,0.000000,0.000000,79.000000,7.900000,1011.600000
50%,15.400000,9.400000,21.200000,0.000000,0.000000,197.000000,11.300000,1015.400000
75%,22.500000,17.200000,28.900000,1.000000,0.000000,297.000000,15.700000,1019.700000
max,42.200000,37.200000,50.000000,571.500000,1780.000000,360.000000,78.300000,1051.000000


In [16]:
weather_df.isnull().sum()

time          0
tavg          0
tmin          0
tmax          0
prcp          0
snow          0
wdir          0
wspd          0
pres          0
airport_id    0
dtype: int64

#### Use data from Los Angeles International Airport (LAX) for this project

In [17]:
lax_weather = weather_df[weather_df['airport_id'] == 'LAX']
lax_weather.shape

(365, 10)

### Merge two datasets

In [18]:
lax_weather = lax_weather.rename(columns={'airport_id': 'dep_airport'})

In [19]:
lax_weather = lax_weather.rename(columns={'time': 'flightdate'})

In [20]:
lax_merged = pd.merge(lax_df, lax_weather, on=['flightdate', 'dep_airport'], how='left')

print(lax_merged.head())

   flightdate  day_of_week                 airline tail_number dep_airport  \
0  2023-01-01            7  American Airlines Inc.      N101NN         LAX   
1  2023-01-02            1  American Airlines Inc.      N116AN         LAX   
2  2023-01-03            2  American Airlines Inc.      N107NN         LAX   
3  2023-01-04            3  American Airlines Inc.      N102NN         LAX   
4  2023-01-05            4  American Airlines Inc.      N110AN         LAX   

      dep_cityname deptime_label  dep_delay  dep_delay_tag dep_delay_type  \
0  Los Angeles, CA       Morning         -2              0      Low <5min   
1  Los Angeles, CA       Morning         -6              0      Low <5min   
2  Los Angeles, CA       Morning         -2              0      Low <5min   
3  Los Angeles, CA       Morning         -3              0      Low <5min   
4  Los Angeles, CA       Morning         -3              0      Low <5min   

   ... model aicraft_age  tavg  tmin  tmax  prcp  snow   wdir  wspd 

In [21]:
lax_merged.shape

(192260, 32)